In [3]:
### Preprocessing 

import scipy.io as sio
import numpy as np
import pandas as pd
import pickle
import os

# Set up data directories
data_dir = './data_ephys/'
behav_dir = './data_behav/'

# Find available subjects
subjects = []
for file in os.listdir(data_dir):
    if file.endswith('.mat') and file.startswith('s'):
        subject_id = file.split('_')[0]
        subjects.append(subject_id)

# Select target subject
target_subject = 's01'

# Load electrophysiology data
ephys_file = f'{data_dir}{target_subject}_ofc_hg_events.mat'
data = sio.loadmat(ephys_file)

# Load main data matrices
game_data = data['game_events_hg']  # nTrials × nTimePoints × nElectrodes
button_data = data['buttonpress_events_hg']  # nTrials × nTimePoints × nElectrodes

# Load behavioral data
csv_file = f'{behav_dir}gamble.data.{target_subject}.csv'
trial_data = pd.read_csv(csv_file)

def find_task_responsive_electrodes(data_matrix, percentile_threshold=75):
    """Find electrodes that show strong task-related activity"""
    n_electrodes = data_matrix.shape[2]
    electrode_responsiveness = []
    
    for e in range(n_electrodes):
        electrode_data = data_matrix[:, :, e]
        avg_response = np.mean(electrode_data, axis=0)
        response_variance = np.var(avg_response)
        
        electrode_responsiveness.append({
            'electrode': e,
            'variance': response_variance,
            'mean_activity': np.mean(electrode_data)
        })
    
    # Sort by responsiveness and get top electrodes
    electrode_responsiveness.sort(key=lambda x: x['variance'], reverse=True)
    threshold = np.percentile([e['variance'] for e in electrode_responsiveness], percentile_threshold)
    responsive_electrodes = [e for e in electrode_responsiveness if e['variance'] >= threshold]
    
    return responsive_electrodes

def extract_electrode_timeseries(data_matrix, electrode_indices):
    """Extract time series for selected electrodes"""
    selected_data = data_matrix[:, :, electrode_indices]
    concatenated_data = selected_data.reshape(-1, len(electrode_indices))
    
    return concatenated_data

def preprocess_timeseries(timeseries_data):
    """Basic preprocessing for DCM"""
    # Remove DC component and normalize
    timeseries_centered = timeseries_data - np.mean(timeseries_data, axis=0)
    timeseries_normalized = (timeseries_centered - np.mean(timeseries_centered, axis=0)) / np.std(timeseries_centered, axis=0)
    
    return timeseries_normalized

# Find responsive electrodes and select top N
game_responsive = find_task_responsive_electrodes(game_data)
button_responsive = find_task_responsive_electrodes(button_data)

# Select top 4 most responsive electrodes
n_electrodes = 4
selected_electrodes = [e['electrode'] for e in game_responsive[:n_electrodes]]

# Extract and preprocess time series
game_timeseries = extract_electrode_timeseries(game_data, selected_electrodes)
button_timeseries = extract_electrode_timeseries(button_data, selected_electrodes)

game_processed = preprocess_timeseries(game_timeseries)
button_processed = preprocess_timeseries(button_timeseries)

# Create and save final data structure for DCM
dcm_data = {
    'subject_id': target_subject,
    'selected_electrodes': selected_electrodes,
    'game_locked': {'Y': game_processed, 'dt': 1/1000},
    'button_locked': {'Y': button_processed, 'dt': 1/1000},
    'behavioral_data': trial_data
}

with open(f'dcm_ready_data_{target_subject}.pkl', 'wb') as f:
    pickle.dump(dcm_data, f)

print(f"DCM data saved for {target_subject}: {len(selected_electrodes)} electrodes, shapes {game_processed.shape}/{button_processed.shape}")

DCM data saved for s01: 2 electrodes, shapes (540180, 2)/(540180, 2)


In [5]:
### Step 2
import numpy as np
import pandas as pd
import pickle

# Load processed data
with open('dcm_ready_data_s01.pkl', 'rb') as f:
    dcm_data = pickle.load(f)

behavioral_data = dcm_data['behavioral_data']

def create_input_timeseries(behavioral_data, neural_length, sampling_rate=1000):
    """Create DCM input matrix aligned with neural data"""
    # Initialize input matrix: time × inputs
    U = np.zeros((neural_length, 3))
    
    # Parameters
    trial_length = 3001   # 3001 time points per trial
    
    # Get valid trials (exclude timeouts)
    valid_trials = behavioral_data[behavioral_data['choice.class'] != 'Timeout'].copy()
    valid_trials = valid_trials.reset_index(drop=True)
    
    # Create inputs for each trial
    for trial_idx, trial in valid_trials.iterrows():
        if trial_idx >= neural_length // trial_length:
            break  # Don't exceed neural data length
            
        # Calculate trial start in neural data
        trial_start = trial_idx * trial_length
        trial_end = trial_start + trial_length
        
        if trial_end > neural_length:
            break
            
        # INPUT 1: Game onset (driving input)
        game_onset_sample = trial_start + 750  # Game presents at 750ms
        if game_onset_sample < neural_length:
            U[game_onset_sample, 0] = 1.0
        
        # INPUT 2: Choice type (modulatory - sustained)
        choice_value = 1.0 if trial['choice.class'] == 'Gamble' else 0.0
        choice_start = trial_start + 750
        choice_duration = 2000  # Up to 2s choice period
        choice_end = min(choice_start + choice_duration, trial_end)
        U[choice_start:choice_end, 1] = choice_value
        
        # INPUT 3: Outcome (modulatory - transient)
        outcome_value = 1.0 if 'Won' in str(trial['outcome']) else 0.0
        outcome_sample = choice_start + 550  # Outcome at ~1.3s after choice
        outcome_duration = 500  # 500ms outcome processing
        outcome_end = min(outcome_sample + outcome_duration, trial_end)
        
        if outcome_sample < neural_length:
            U[outcome_sample:outcome_end, 2] = outcome_value
    
    return U

# Create the input matrix
U = create_input_timeseries(behavioral_data, dcm_data['game_locked']['Y'].shape[0])

# Create final DCM input structure
dcm_inputs = {
    'U': U,  # Input matrix (time × inputs)
    'dt': 1/1000,  # Sampling interval
    'input_names': ['GameOnset', 'ChoiceType', 'Outcome'],
    'input_descriptions': {
        'GameOnset': 'Driving input - game presentation onset',
        'ChoiceType': 'Modulatory input - choice context (gamble vs safe)',
        'Outcome': 'Modulatory input - outcome valence (win vs loss)'
    },
    'input_types': ['driving', 'modulatory', 'modulatory']
}

# Add to DCM data structure and save
dcm_data['inputs'] = dcm_inputs

with open('dcm_ready_data_s01.pkl', 'wb') as f:
    pickle.dump(dcm_data, f)

print(f"DCM inputs created: {U.shape}, {np.sum(U[:,0])} game onsets, ready for network design")

DCM inputs created: (540180, 3), 180.0 game onsets, ready for network design


In [7]:
### Step 3
import numpy as np
import pickle

def estimate_connectivity_from_data(Y, method='correlation'):
    """Estimate initial connectivity strengths from neural data"""
    
    if method == 'correlation':
        # Simple cross-correlation approach
        corr_forward = np.corrcoef(Y[:-1, 0], Y[1:, 1])[0, 1]  # E3(t) → E1(t+1)
        corr_backward = np.corrcoef(Y[:-1, 1], Y[1:, 0])[0, 1]  # E1(t) → E3(t+1)
        
        # Scale correlations to reasonable connection strengths
        A_forward = np.clip(abs(corr_forward) * 1.5, 0.1, 0.9)
        A_backward = np.clip(abs(corr_backward) * 1.2, 0.1, 0.7)
        
    elif method == 'adaptive':
        # Adaptive based on signal characteristics
        signal_var = np.var(Y, axis=0)
        signal_ratio = signal_var[0] / signal_var[1]  # E3/E1 variance ratio
        cross_corr = np.corrcoef(Y[:, 0], Y[:, 1])[0, 1]
        
        # Adaptive logic based on which electrode is more active
        if signal_ratio > 1.2:  # E3 more variable → likely upstream
            A_forward = min(0.9, abs(cross_corr) * 1.5)   # Strong E3→E1
            A_backward = min(0.6, abs(cross_corr) * 1.0)  # Moderate E1→E3
        else:  # More balanced
            A_forward = min(0.7, abs(cross_corr) * 1.2)
            A_backward = min(0.7, abs(cross_corr) * 1.2)
    
    return A_forward, A_backward

def estimate_task_specific_connectivity(Y, U, trial_length=3001):
    """Estimate connectivity during choice phase (most relevant for decision-making)"""
    
    # Define choice phase (in samples)
    choice_start, choice_end = 750, 2750
    
    n_trials = Y.shape[0] // trial_length
    correlations_forward = []
    correlations_backward = []
    
    for trial in range(min(n_trials, 20)):  # Use first 20 trials
        trial_start = trial * trial_length
        
        # Extract choice phase data
        y_phase = Y[trial_start + choice_start:trial_start + choice_end, :]
        
        if len(y_phase) > 10:  # Ensure enough data points
            corr_f = np.corrcoef(y_phase[:-1, 0], y_phase[1:, 1])[0, 1]
            corr_b = np.corrcoef(y_phase[:-1, 1], y_phase[1:, 0])[0, 1]
            
            if not np.isnan(corr_f): correlations_forward.append(corr_f)
            if not np.isnan(corr_b): correlations_backward.append(corr_b)
    
    # Average across trials and scale to reasonable connection strengths
    avg_forward = np.mean(correlations_forward) if correlations_forward else 0
    avg_backward = np.mean(correlations_backward) if correlations_backward else 0
    
    A_forward = np.clip(abs(avg_forward) * 1.3, 0.2, 0.8)
    A_backward = np.clip(abs(avg_backward) * 1.1, 0.2, 0.6)
    
    return A_forward, A_backward

def create_dcm_matrices(Y, U):
    """Create A, B, and C matrices for 2-electrode DCM with data-driven connectivity"""
    
    # Estimate connectivity using multiple methods
    A_forward_1, A_backward_1 = estimate_connectivity_from_data(Y, method='correlation')
    A_forward_2, A_backward_2 = estimate_connectivity_from_data(Y, method='adaptive')
    A_forward_3, A_backward_3 = estimate_task_specific_connectivity(Y, U)
    
    # Combine estimates (weighted average)
    A_forward = (A_forward_1 * 0.3 + A_forward_2 * 0.4 + A_forward_3 * 0.3)
    A_backward = (A_backward_1 * 0.3 + A_backward_2 * 0.4 + A_backward_3 * 0.3)
    
    # Create A matrix with data-driven values
    A_prior = np.array([
        [-1.0, A_backward],    
        [A_forward, -1.0]      
    ])
    
    # C matrix (driving inputs)
    C_prior = np.array([
        [1.0, 0.0, 0.0],  # Game onset drives E3 strongly
        [0.5, 0.0, 0.0]   # Game onset drives E1 moderately
    ])
    
    # B matrices (modulatory effects)
    B_prior = np.zeros((3, 2, 2))
    B_prior[1, 1, 0] = 0.3  # Choice type modulates E3→E1
    B_prior[2, 0, 0] = 0.2  # Outcome modulates E3 self-connection
    B_prior[2, 1, 1] = 0.2  # Outcome modulates E1 self-connection
    
    return A_prior, B_prior, C_prior, A_forward, A_backward

# Load data and create DCM matrices
with open('dcm_ready_data_s01.pkl', 'rb') as f:
    dcm_data = pickle.load(f)

Y = dcm_data['game_locked']['Y']
U = dcm_data['inputs']['U']

# Create matrices with data-driven connectivity
A, B, C, forward_strength, backward_strength = create_dcm_matrices(Y, U)

# Add architecture to DCM data
dcm_data['architecture'] = {
    'A': A,  # Intrinsic connections
    'B': B,  # Modulatory connections  
    'C': C,  # Driving connections
    'connectivity_strengths': {
        'E3_to_E1': forward_strength,
        'E1_to_E3': backward_strength
    }
}

# Save complete DCM specification
with open('dcm_ready_data_s01.pkl', 'wb') as f:
    pickle.dump(dcm_data, f)

print(f"DCM architecture complete: E3→E1={forward_strength:.3f}, E1→E3={backward_strength:.3f}, ready for model specification")

DCM architecture complete: E3→E1=0.285, E1→E3=0.264, ready for model specification


In [9]:
import numpy as np
import pickle

# Load DCM data
with open('dcm_ready_data_s01.pkl', 'rb') as f:
    dcm_data = pickle.load(f)

def setup_dcm_priors():
    """Set up prior distributions for DCM parameters (ECoG-optimized)"""
    return {
        'A': {
            'mean': np.array([[-0.5,  0.0], [ 0.0, -0.5]]),
            'variance': 0.25 * np.ones((2, 2))
        },
        'B': {
            'mean': np.zeros((3, 2, 2)),
            'variance': 0.1 * np.ones((3, 2, 2))
        },
        'C': {
            'mean': np.array([[0.5, 0.0, 0.0], [0.2, 0.0, 0.0]]),
            'variance': 0.5 * np.ones((2, 3))
        },
        'temporal_scaling': {'mean': 2.0, 'variance': 0.5},  # Faster dynamics for ECoG
        'noise': {'precision_prior': 1.0, 'precision_shape': 1.0}
    }

def setup_parameter_constraints():
    """Define which parameters are free vs fixed"""
    return {
        'A_free': np.array([[True,  True], [True,  True]]),
        'B_free': np.array([
            [[False, False], [False, False]],  # B1: No modulation by driving input
            [[False, True],  [False, False]],  # B2: Choice modulates E3→E1 only  
            [[True,  False], [False, True]]    # B3: Outcome modulates self-connections
        ]),
        'C_free': np.array([[True, False, False], [True, False, False]]),
        'stability_constraint': True,
        'connection_threshold': 0.05
    }

def dcm_forward_model(params, U, dt=0.001):
    """
    Forward model: simulate neural responses given parameters and inputs
    Core DCM equation: dz/dt = (A + Σ u_j B_j)z + Cu
    """
    A, B, C = params['A'], params['B'], params['C']
    n_time, n_inputs = U.shape
    n_regions = A.shape[0]
    
    # Initialize state
    z = np.zeros((n_time, n_regions))
    
    # Integrate over time using Euler method
    for t in range(1, n_time):
        z_curr = z[t-1, :]
        u_curr = U[t-1, :]
        
        # Effective connectivity matrix: A + Σ u_j B_j
        A_eff = A.copy()
        for j in range(n_inputs):
            A_eff += u_curr[j] * B[j]
        
        # State derivative: dz/dt = A_eff * z + C * u
        dzdt = A_eff @ z_curr + C @ u_curr
        
        # Euler integration
        z[t, :] = z_curr + dt * dzdt
    
    return z

def create_complete_dcm_spec():
    """Create complete DCM specification ready for estimation"""
    
    # Extract data
    Y = dcm_data['game_locked']['Y']
    U = dcm_data['inputs']['U']
    
    # Get priors and constraints
    priors = setup_dcm_priors()
    constraints = setup_parameter_constraints()
    
    return {
        # Data
        'Y': Y,
        'U': U,
        'dt': 0.001,
        
        # Model structure  
        'A': dcm_data['architecture']['A'],
        'B': dcm_data['architecture']['B'],
        'C': dcm_data['architecture']['C'],
        
        # Priors and constraints
        'priors': priors,
        'constraints': constraints,
        
        # Model properties
        'n_regions': 2,
        'n_inputs': 3,
        'n_timepoints': Y.shape[0],
        'region_names': ['Electrode_3', 'Electrode_1'],
        'input_names': ['GameOnset', 'ChoiceType', 'Outcome'],
        
        # Forward model and estimation settings
        'forward_model': dcm_forward_model,
        'estimation': {
            'algorithm': 'variational_bayes',
            'max_iterations': 128,
            'convergence_threshold': 1e-4,
            'initialization': 'prior_mean'
        }
    }

# Create and save complete specification
dcm_complete = create_complete_dcm_spec()
dcm_data['dcm_specification'] = dcm_complete

with open('dcm_ready_data_s01.pkl', 'wb') as f:
    pickle.dump(dcm_data, f)

# Count free parameters
constraints = dcm_complete['constraints']
n_free_params = (np.sum(constraints['A_free']) + 
                 np.sum(constraints['B_free']) + 
                 np.sum(constraints['C_free']))

print(f"DCM specification complete: {dcm_complete['n_timepoints']} timepoints, "
      f"{n_free_params} free parameters, ready for estimation")

DCM specification complete: 540180 timepoints, 9 free parameters, ready for estimation


In [13]:
import numpy as np
from scipy.optimize import minimize
from scipy import linalg
import pickle

# Load complete DCM specification
with open('dcm_ready_data_s01.pkl', 'rb') as f:
    dcm_data = pickle.load(f)

dcm_spec = dcm_data['dcm_specification']

def unpack_parameters(theta, constraints, priors):
    """Convert parameter vector back to A, B, C matrices"""
    A = priors['A']['mean'].copy()
    B = priors['B']['mean'].copy()  
    C = priors['C']['mean'].copy()
    
    # Unpack parameter vector
    idx = 0
    
    # A parameters
    n_A = np.sum(constraints['A_free'])
    A[constraints['A_free']] = theta[idx:idx+n_A]
    idx += n_A
    
    # B parameters
    n_B = np.sum(constraints['B_free'])
    B[constraints['B_free']] = theta[idx:idx+n_B]
    idx += n_B
    
    # C parameters
    n_C = np.sum(constraints['C_free'])
    C[constraints['C_free']] = theta[idx:idx+n_C]
    
    return A, B, C

def log_likelihood(theta, Y_obs, U, priors, constraints):
    """Compute log-likelihood of parameters given data"""
    # Check for invalid parameters first
    if np.any(~np.isfinite(theta)) or np.any(np.abs(theta) > 5.0):
        return -1e10  # Return large negative value instead of -inf
    
    try:
        # Unpack parameters
        A, B, C = unpack_parameters(theta, constraints, priors)
        
        # Enhanced stability check
        eigenvals = np.real(linalg.eigvals(A))
        if np.any(eigenvals > -0.05) or np.any(~np.isfinite(eigenvals)):
            return -1e10  # Unstable system
        
        # Simulate response using forward model (use smaller subset for stability)
        n_subset = min(10000, Y_obs.shape[0])  # Reduced for stability
        Y_pred = dcm_spec['forward_model']({'A': A, 'B': B, 'C': C}, 
                                         U[:n_subset, :])
        
        # Check for NaN/inf in predictions
        if np.any(~np.isfinite(Y_pred)):
            return -1e10
        
        # Compute log-likelihood with regularization
        residuals = Y_obs[:n_subset, :] - Y_pred
        
        # Robust noise estimation
        sigma2 = max(0.01, np.var(residuals) + 1e-6)  # Prevent zero variance
        
        # Regularized log-likelihood
        mse = np.mean(residuals**2)
        if not np.isfinite(mse):
            return -1e10
            
        log_lik = -0.5 * mse / sigma2 * n_subset - 0.5 * n_subset * np.log(2*np.pi*sigma2)
        
        # Regularized priors (prevent division by zero)
        log_prior = 0.0
        
        # A prior with regularization
        A_residual = A - priors['A']['mean']
        A_var_reg = priors['A']['variance'][constraints['A_free']] + 1e-6
        log_prior += -0.5 * np.sum((A_residual[constraints['A_free']])**2 / A_var_reg)
        
        # B prior with regularization  
        B_residual = B - priors['B']['mean']
        B_var_reg = priors['B']['variance'][constraints['B_free']] + 1e-6
        log_prior += -0.5 * np.sum((B_residual[constraints['B_free']])**2 / B_var_reg)
        
        # C prior with regularization
        C_residual = C - priors['C']['mean']
        C_var_reg = priors['C']['variance'][constraints['C_free']] + 1e-6
        log_prior += -0.5 * np.sum((C_residual[constraints['C_free']])**2 / C_var_reg)
        
        total_log_lik = log_lik + log_prior
        
        # Final check for validity
        if not np.isfinite(total_log_lik):
            return -1e10
            
        return total_log_lik
        
    except:
        return -1e10

def estimate_dcm_parameters(Y_obs, U, priors, constraints, method='L-BFGS-B'):
    """Estimate DCM parameters using maximum a posteriori (MAP)"""
    
    # Initial parameter vector (start from prior means with small perturbation)
    theta_init = np.concatenate([
        priors['A']['mean'][constraints['A_free']],
        priors['B']['mean'][constraints['B_free']],  
        priors['C']['mean'][constraints['C_free']]
    ])
    
    # Add small random perturbation to avoid local minima
    theta_init += np.random.normal(0, 0.01, len(theta_init))
    
    # Define objective function (negative log-posterior)
    def objective(theta):
        return -log_likelihood(theta, Y_obs, U, priors, constraints)
    
    # Tighter parameter bounds to prevent instability
    bounds = [(-1.5, 1.5) for _ in range(len(theta_init))]
    
    # More conservative optimization settings
    result = minimize(objective, theta_init, method=method, bounds=bounds,
                     options={
                         'maxiter': 50,     # Reduced iterations
                         'ftol': 1e-6,      # Function tolerance
                         'gtol': 1e-5,      # Gradient tolerance  
                         'disp': False,
                         'maxfun': 1000     # Max function evaluations
                     })
    
    return result, theta_init

def finalize_dcm_results(result, theta_init, priors, constraints):
    """Extract and organize final DCM results"""
    
    # Get final parameters
    theta_final = result.x if result.success else theta_init
    A_final, B_final, C_final = unpack_parameters(theta_final, constraints, priors)
    
    # Create results summary
    dcm_results = {
        'parameters': {
            'A': A_final,
            'B': B_final, 
            'C': C_final,
            'theta_vector': theta_final
        },
        'estimation': {
            'success': result.success,
            'log_likelihood': -result.fun if result.success else None,
            'iterations': result.nit if hasattr(result, 'nit') else None,
            'method': 'MAP estimation'
        },
        'connectivity_summary': {
            'E3_to_E1': A_final[1, 0],
            'E1_to_E3': A_final[0, 1],
            'E3_self': A_final[0, 0],
            'E1_self': A_final[1, 1]
        }
    }
    
    return dcm_results

# Extract data
Y_obs = dcm_spec['Y']
U = dcm_spec['U']
priors = dcm_spec['priors']
constraints = dcm_spec['constraints']

# Use conservative computational subset for stability
subsample_factor = max(10, Y_obs.shape[0] // 10000)  # Ensure <= 10k points for stability
Y_subset = Y_obs[::subsample_factor, :]
U_subset = U[::subsample_factor, :]

# Additional data validation
if np.any(~np.isfinite(Y_subset)) or np.any(~np.isfinite(U_subset)):
    print("Warning: Data contains NaN/inf values, attempting to clean...")
    Y_subset = np.nan_to_num(Y_subset, nan=0.0, posinf=1.0, neginf=-1.0)
    U_subset = np.nan_to_num(U_subset, nan=0.0, posinf=1.0, neginf=-1.0)

# Run estimation
result, theta_init = estimate_dcm_parameters(Y_subset, U_subset, priors, constraints)

# Finalize results
dcm_results = finalize_dcm_results(result, theta_init, priors, constraints)

# Save results
dcm_data['dcm_results'] = dcm_results

with open('dcm_ready_data_s01.pkl', 'wb') as f:
    pickle.dump(dcm_data, f)

# Report results
connectivity = dcm_results['connectivity_summary']
print(f"DCM estimation complete (stabilized): E3→E1={connectivity['E3_to_E1']:.3f}, "
      f"E1→E3={connectivity['E1_to_E3']:.3f}, "
      f"success={dcm_results['estimation']['success']}, "
      f"using {Y_subset.shape[0]} datapoints")

DCM estimation complete (stabilized): E3→E1=-0.006, E1→E3=-0.002, success=True, using 10004 datapoints
